## Homework 10 — Kubernetes

Answers to the questions from `homework.md`.

Note: question 1 was not executed directly in this environment because the `scikit-learn` library is missing; the value was inferred from the deployment homework (same model).


### Q1. Result of `q6_test.py`

In [5]:
!python ../05-deployment/homework/q6_test.py


{'conversion_probability': 0.49999999999842815, 'conversion': False}


### Q2. kind version

In [9]:
!kind --version

kind version 0.30.0


### Q3. Smallest deployable unit

The smallest compute unit that we can create and manage in Kubernetes is the **Pod**.


### Q4. Type of the initial service

The Kubernetes service created by default in the cluster (service `kubernetes` in the `default` namespace) is of type **ClusterIP**.


### Q5. Loading the Docker image into kind

To register the local image `zoomcamp-model:3.13.10-hw10` in the kind cluster, we use:

In [12]:
!kind load docker-image zoomcamp-model:3.13.10-hw10


Image: "zoomcamp-model:3.13.10-hw10" with ID "sha256:a9fafc8a64dcfe3411577b44432c1bb5907b8e929ad32867d9c99f45053b4e0b" not yet present on node "kind-control-plane", loading...


### Q6. Container port

In the deployment, the FastAPI service exposes port **9696** (see `uvicorn` in `q6_predict.py` and the `containerPort` field in `deployment.yaml`).

Answer: **9696**.


In [18]:
!kubectl apply -f deployment.yaml
!kubectl get pods
!kubectl describe deployment subscription | grep -i port -A2

deployment.apps/subscription unchanged
NAME                            READY   STATUS             RESTARTS      AGE
subscription-58778cfcfd-hztpq   0/1     CrashLoopBackOff   7 (81s ago)   17m
    Port:       9696/TCP
    Host Port:  0/TCP
    Limits:
      cpu:     100m


In [21]:
# Q6 helper: apply the deployment and inspect pods
# Run this cell from the repo root with a Kubernetes cluster created by kind.

!kubectl apply -f deployment.yaml
!kubectl get pods
!kubectl describe deployment subscription | grep -i 'Port' -A2


deployment.apps/subscription unchanged
NAME                            READY   STATUS             RESTARTS       AGE
subscription-58778cfcfd-hztpq   0/1     CrashLoopBackOff   7 (2m2s ago)   18m
    Port:       9696/TCP
    Host Port:  0/TCP
    Limits:
      cpu:     100m


### Q7. Service selector

The deployment uses the label `app: subscription`, so the Service selector must be:

```yaml
selector:
  app: subscription
```

**How I tested Q7**

1. Apply the Service manifest:

```bash
kubectl apply -f cohorts/2025/10-kubernetes/service.yaml
kubectl get svc
```

   You should see the `subscription` service, type `LoadBalancer`.

2. Check that the selector points to the pod (endpoints are not empty):

```bash
kubectl get endpoints subscription
```

   There should be at least one IP with port `9696`.

3. Full end-to-end test (Service + port):

   - In one terminal:

```bash
kubectl port-forward service/subscription 9696:80
```

   - In another terminal (from the repo root):

```bash
python cohorts/2025/05-deployment/homework/q6_test.py
```

   If the script returns a dictionary with `conversion_probability` and no connection error, then the Service selector and `targetPort` are correct.

Answer: the value that goes in `<???>` is **`subscription`**.


In [25]:
!kubectl apply -f service.yaml
!kubectl get svc
!kubectl get endpoints subscription
!kubectl port-forward service/subscription 9696:80
!python ../05-deployment/homework/q6_test.py


service/subscription unchanged
NAME           TYPE           CLUSTER-IP    EXTERNAL-IP   PORT(S)        AGE
kubernetes     ClusterIP      10.96.0.1     <none>        443/TCP        53m
subscription   LoadBalancer   10.96.28.22   <pending>     80:31522/TCP   20m
NAME           ENDPOINTS   AGE
subscription               20m
Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696


OSError: [Errno 5] Input/output error

### Q8 (optional). Maximum number of replicas in the HPA

The HPA creation command limits the maximum number of replicas to 3 (`--max=3`).
Therefore, during the load test, the maximum number of replicas you can observe is **3**.
